In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
data.head(5)

In [ ]:
data['date'] = pd.to_datetime(data['date'])

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

data['CPM'] = data.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)

In [ ]:
data = data[data['CPM'] >= 0] # Мы же и там и там убираем отрицательные значения? Было бы странно убрать только из теста.

In [ ]:
data.drop(['integration_type_id','total_revenue','measurable_impressions',
           'viewable_impressions','revenue_share_percent', 'total_impressions'], axis=1, inplace=True)

In [ ]:
date_for_split = pd.to_datetime('2019-06-22')

In [ ]:
train = data[data.date < date_for_split]
test = data[data.date >= date_for_split]

In [ ]:
train = train[train['CPM'] <= np.percentile(train['CPM'], 95)]
test= test[test['CPM'] <= np.percentile(test['CPM'], 95)]

In [ ]:
X_train = train.drop('CPM', axis=1)
X_test = test.drop('CPM', axis=1)
y_train = train.CPM.values
y_test = test.CPM.values

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
X_train

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
model = CatBoostRegressor(random_seed=42, verbose=0, task_type = 'GPU', iterations = 2000)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_pred, y_test)